# WELCOME TO ATTEMPT **3** of **"*THE MOST AUTISTIC THING I HAVE DONE, LIKE, EVER*"**

This is mostly attempt 2 again, **note to self, backups are not optional**.
The whole attempt 2/3 is due to v1's failings, but a bit of a recap first.

## Recap
This whole endevour is to check definitely how much time has passed in the webserial *"Wearing Power Armor to a Magic School"* written by **JCB112** (or `WPatMS` for short), the other goal is to predict in which chapter a *"communication deadline"* which happens 5 weeks after the beginning of the book.

Thanks to a **TON** of manual analysis we know that the book can't start any earier than 15:30 on the first day

## V1 Sucesses
As much as V1 was flawed from the start it did a bit of good:
1. **THE GRAPHS**, v1 gave us some very good graphs, even though they were a bit wonky in places *(the X-axis)*
2. It helped greately with the manual analysis
3. It proved that this way of parsing the timestamps can work

## V1 Problems
This is a bit easier than the above section:
1. The creation of html files and only then parsing them is a stupid idea. It would have some merit, if any changes would be made for these files, but such a thing was just not done
2. The jupyter notebook changed some variables in a cell, which did not have the definition, this made the code require a full restart on every run, as running it twice changed some variables
3. The code was not portable, the creation of the html files and the folders, required the use of some linux only commands, i want to change that by ... just not having any files saved
4. **THE GRAPHS™** all have a wonky x-axes, this is due to x being "event index" and not "chapter". This was easier, but it makes everything harder to understand

## V2 Failings
Ok this will be short. ***BACKUPS***.

So I opened a vscode server, connect to it from school (this is after *4 fucking days of coding*) **AND IT FUCKING DELETES THE WHOLE FUCKING FILE**

As to the fix, this is a git repo.

The second main failing was keeping the use of timestamps, and not just "minutes after midnight".

## V3 - Functional description

1. Parse the epub (it will be XHTML), as a list called `book` containing `chapters` that are a list of lines
2. Search for timestamps
3. Convert the timestamps to "minutes after midnight
4. Do some graphs
5. Find the amount of days that passed
6. Find the chapter, where the deadline passes